Questions,

https://www.kaggle.com/c/house-prices-advanced-regression-techniques

DataDescription

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

Missing Value and ordinal encoding

https://www.kaggle.com/mitramir5/missing-values-ordinal-data-and-stories/notebook

In [2]:
import pandas as pd
import numpy as np


import sklearn
import warnings

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# from sklearn.ensemble import GradientBoostingRegressor
# from lightgbm import LGBMRegressor
# from sklearn.linear_model import Lasso
# from catboost import CatBoostRegressor
# from xgboost import XGBRegressor

# Split the data into test data and train data

In [5]:
df_original=pd.read_csv("../010_data/train.csv")

In [3]:
df_original.shape

(1460, 81)

In [4]:
temp=pd.qcut(df_original['SalePrice'],[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-100'])

X_train, X_test, _, _ = train_test_split(temp, temp, test_size=0.2, random_state=42,stratify =temp)


In [5]:
df_train=df_original.loc[X_train.index].copy()
df_test=df_original.loc[X_test.index].copy()

del df_original

In [6]:
temp=pd.qcut(df_train['SalePrice'],[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-100'])

X_train, X_test, _, _ = train_test_split(temp, temp, test_size=0.3, random_state=42,stratify =temp)

In [7]:
df_dev=df_train.loc[X_test.index].copy()
df_train=df_train.loc[X_train.index].copy()


In [8]:
# df_train.to_csv('../010_data/W207_train.csv',index=False)
# df_train.to_csv('../010_data/W207_dev.csv',index=False)
# df_test.to_csv('../010_data/W207_test.csv',index=False)

# Import the pre-splited data

In [9]:
df_train=pd.read_csv('../010_data//W207_train.csv')
df_train=pd.read_csv('../010_data//W207_dev.csv')
df_test=pd.read_csv('../010_data/W207_test.csv')

# EDA

including the missing value conformation only us the df_train data please

https://www.kaggle.com/dgawlik/house-prices-eda

In [10]:
(df_train.isnull().sum()/df_train.shape[0]).sort_values(ascending=False)[:20]

PoolQC          0.995104
MiscFeature     0.957160
Alley           0.938800
Fence           0.806610
FireplaceQu     0.484700
LotFrontage     0.197062
GarageType      0.055080
GarageCond      0.055080
GarageYrBlt     0.055080
GarageFinish    0.055080
GarageQual      0.055080
BsmtExposure    0.030600
BsmtFinType1    0.029376
BsmtCond        0.029376
BsmtQual        0.029376
BsmtFinType2    0.029376
MasVnrArea      0.007344
MasVnrType      0.007344
Exterior2nd     0.000000
Exterior1st     0.000000
dtype: float64

# Data Cleancing

# Miss value handling and Value conversion

In [11]:
col_drop=['Id','PoolQC','MiscFeature','Alley','Fence','FireplaceQu','LotFrontage']

col_cat=['MSZoning','Street','LandContour','Utilities','LotConfig','LandSlope','Neighborhood',
        'Condition1','Condition2', 'BldgType',
       'HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating',
         'CentralAir','Electrical','GarageType','SaleType','SaleCondition'
        ]

col_order=['MSSubClass','ExterQual','LotShape','ExterCond',
           'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
          'HeatingQC','Functional','KitchenQual','GarageQual','GarageFinish','GarageCond','PavedDrive',
          ]

col_num=['OverallQual', 'OverallCond','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea','MiscVal','YearBuilt','YearRemodAdd','MoSold','YrSold','GarageYrBlt']

col_target=['SalePrice']

# check the integrety
# temp=col_cat+col_order+col_num
# [i for i in temp if i in col_drop]

In [12]:
# categorical data

cat_values={
    'MSZoning': ['None','A','C','FV','I','RH','RL', 'RP' , 'RM'],
    'Street': ['None', 'Pave', 'Grvl'],
    'LandContour': ['None', 'Lvl', 'Bnk', 'HLS', 'Low'],
    'Utilities': ['None', 'AllPub', 'NoSewr', 'NoSeWa', 'ELO'],
    'LotConfig': ['None', 'Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'],
    'LandSlope': ['None', 'Gtl', 'Mod', 'Sev'],
    'Neighborhood': ["Blmngtn","Heights","Bluestem","Briardale",
                     "Brookside","Clear","CollgCr","Crawfor",
                     "Edwards","Gilbert","IDOTRR","MeadowV","Mitchel","Names",
                     "NoRidge","NPkVill","NridgHt","NWAmes","SWISU","Sawyer",
                     "SawyerW","Somerst","StoneBr","Veenker"
                    ],
    'Condition1': ['None',  'Artery', 'Feedr',  'Norm',
                'RRNn',  'RRAn', 'PosN', 'PosA',
                'RRNe',  'RRAe'],
    'Condition2':  ['None',  'Artery', 'Feedr',  'Norm',
                'RRNn',  'RRAn', 'PosN', 'PosA',
                'RRNe',  'RRAe'],
    'BldgType': ['None', '1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'TwnhsI'],
    'HouseStyle': ['None',  '1Story',  '1.5Fin',  '1.5Unf', '2Story',  '2.5Fin',  '2.5Unf',  'SFoyer',  'SLvl'],
    'RoofStyle': ['None', 'Gable', 'Hip', 'Gambrel', 'Mansard', 'Flat', 'Shed'],
    'RoofMatl': ['None',  'ClyTile', 'CompShg',  'Membran',  'Metal',  'Roll', 'Tar&Grv',  'WdShake',  'WdShngl'],
    'Exterior1st': ['None',  'AsbShng',  'AsphShn',  'BrkComm',  'BrkFace',  'CBlock',  'CemntBd',  
                    'ImStucc', 'MetalSd',  'Other',  'Plywood',  'PreCast',  'Stone',  'Stucco',  
                    'Wd Sdng',  'WdShing'],
    'Exterior2nd': ['None',  'AsbShng',  'AsphShn',  'BrkComm',  'BrkFace',  'CBlock',  'CemntBd',  
                    'ImStucc', 'MetalSd',  'Other',  'Plywood',  'PreCast',  'Stone',  'Stucco',  
                    'Wd Sdng',  'WdShing'],
    'MasVnrType': ['None', 'BrkFace', 'Stone', 'BrkCmn'],
    'Foundation': ['None', 'PConc', 'CBlock', 'BrkTil', 'Wood', 'Slab', 'Stone'],
    'Heating': ['None', 'GasA', 'GasW', 'Grav', 'Wall', 'OthW', 'Floor'],
    'CentralAir': ['None', 'Y', 'N'],
    'Electrical': ['None', 'SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix'],
    'GarageType': ['None', 'Attchd', 'Detchd', 'BuiltIn', 'CarPort', 'Basment', '2Types'],
    'SaleType': ['None', 'WD','CWD', 'VWD', 'New', 'COD',
                'ConLD',  'ConLI',  'ConLw',  'Con',  'Oth'],
    'SaleCondition': ['None',  'Normal',  'Abnorml',  'Partial',  'AdjLand',  'Alloca',  'Family'],
    
    #msclass
    'MSSubClass':['None',20,30,40,45,50,60,70,75,80,85, 90,120,150,160,180,190],
    #ExterQual
    'ExterQual':['None','Po','Fa','TA','Gd','Ex'],
    #LotShape
    'LotShape':['None','Reg','IR1' ,'IR2','IR3'],
    #ExterCond
    'ExterCond':['None','Po','Fa','TA','Gd','Ex'],
    #BsmtQual
    'BsmtQual':['None','Fa','TA','Gd','Ex'],
    #BsmtCond
    'BsmtCond':['None','Po','Fa','TA','Gd','Ex'],
    #BsmtExposure
    'BsmtExposure':['None','No','Mn','Av','Gd'],
    #BsmtFinType1
    'BsmtFinType1':['None','Unf','LwQ', 'Rec','BLQ','ALQ' , 'GLQ' ],
    #BsmtFinType2
   'BsmtFinType2':['None','Unf','LwQ', 'Rec','BLQ','ALQ' , 'GLQ' ],
    #HeatingQC
    'HeatingQC':['None','Po','Fa','TA','Gd','Ex'],
    #Functional
   'Functional':['None','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'],
    #FireplaceQu
#     ['None','Po','Fa','TA','Gd','Ex'],
    #KitchenQual
    'KitchenQual':['None','Fa','TA','Gd','Ex'],
    #GarageFinish
    'GarageFinish':['None','Unf','RFn','Fin'],
    #GarageQual
    'GarageQual':['None','Po','Fa','TA','Gd','Ex'],
    #GarageCond
    'GarageCond':['None','Po','Fa','TA','Gd','Ex'],
    #PoolQC
    #['None','Fa','Gd','Ex'],
    #Fence
#     ['None','MnWw','GdWo','MnPrv','GdPrv'] ]
    'PavedDrive':['None','N','P',  'Y']
}

In [13]:
# result={}
# for i in col_cat:
#     result[i]=['None']
#     temp=df_original[i].fiallna().unique()
#     for e in temp:
#         result[i].append(e)
# result

In [14]:
orders_encoding_result={}
for e in col_order:
    temp=cat_values[e]
    orders_encoding_result[e]={}
    for i in range(len(temp)):
        orders_encoding_result[e][temp[i]]=i
orders_encoding_result          

{'MSSubClass': {'None': 0,
  20: 1,
  30: 2,
  40: 3,
  45: 4,
  50: 5,
  60: 6,
  70: 7,
  75: 8,
  80: 9,
  85: 10,
  90: 11,
  120: 12,
  150: 13,
  160: 14,
  180: 15,
  190: 16},
 'ExterQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
 'LotShape': {'None': 0, 'Reg': 1, 'IR1': 2, 'IR2': 3, 'IR3': 4},
 'ExterCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
 'BsmtQual': {'None': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
 'BsmtCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
 'BsmtExposure': {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4},
 'BsmtFinType1': {'None': 0,
  'Unf': 1,
  'LwQ': 2,
  'Rec': 3,
  'BLQ': 4,
  'ALQ': 5,
  'GLQ': 6},
 'BsmtFinType2': {'None': 0,
  'Unf': 1,
  'LwQ': 2,
  'Rec': 3,
  'BLQ': 4,
  'ALQ': 5,
  'GLQ': 6},
 'HeatingQC': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
 'Functional': {'None': 0,
  'Sev': 1,
  'Maj2': 2,
  'Maj1': 3,
  'Mod': 4,
  'Min2': 5,
  'Min1': 6,
  'Typ': 7},
 'KitchenQual':

In [15]:
def ImputerConverter(df,col_num,col_order,col_cat,orders_encoding_result):
    df=df.drop(col_drop,axis=1).copy()
    df[col_num]=df[col_num].fillna(0)
    df[col_order+col_cat]=df[col_order+col_cat].fillna('None')

    for i in col_order:
        df[i]=df[i].map(orders_encoding_result[i])
    return df

df_train_IC=ImputerConverter(df_train,col_num,col_order,col_cat,orders_encoding_result)
df_dev_IC=ImputerConverter(df_dev,col_num,col_order,col_cat,orders_encoding_result)       

In [16]:
(df_train_IC.isnull().sum()/df_train_IC.shape[0]).sort_values(ascending=False)[:10]

SalePrice      0.0
RoofStyle      0.0
Exterior1st    0.0
Exterior2nd    0.0
MasVnrType     0.0
MasVnrArea     0.0
ExterQual      0.0
ExterCond      0.0
Foundation     0.0
BsmtQual       0.0
dtype: float64

# Category column need to encode (One hot?)

# Pipe line

is this needed?

https://www.kaggle.com/alexandrasciocchetti/housing-prediction-w-pipeline-ensemble-methods

In [17]:
#define transform strategy
numerical_transformer = SimpleImputer(missing_values =np.nan,strategy='constant')

#categorical columns are encoded during this step
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [18]:
numerical_cols=col_num+col_order
categorical_cols=col_cat

preprocess = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

def process_and_fit(model, train, target):
    pipeline = Pipeline(steps=[('preprocess', preprocess),
                   ('scale', StandardScaler()),
                   ('model', model)])
    pipeline.fit(train, target)
    return pipeline

def rmse(validation, target):
    return np.sqrt(mean_squared_error(validation, target))

In [19]:
# preprocess.fit_transform(df_train_IC)

In [20]:
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [21]:
# get_feature_names(preprocess)

In [22]:
X_train=df_train_IC[col_num+col_order+col_cat]
y_train=df_train_IC[col_target].to_numpy().ravel()

X_valid=df_dev_IC[col_num+col_order+col_cat]
y_valid=df_dev_IC[col_target].to_numpy().ravel()

In [26]:
randomForest = RandomForestRegressor(n_estimators=1000, random_state=42)


#Random Forest
rfPipeline = process_and_fit(randomForest, X_train, y_train)
rfPred = rfPipeline.predict(X_valid)

# models = ['Random Forest'
# #           , 'Gradient Boosting', 'Cat Boost', 'XGBoost', 'Lasso Regression', 'Light GBM'
#          ]
models = {'Random Forest': rfPred
#                , gbPred, cbPred, xgbPred, lassoPred, gbmPred
          }

avg_score = 0

result={}
for m in models:
    score = rmse(y_valid, models[m])
    result[m]=score

result


{'Random Forest': 29433.47773171397}